In [53]:
import json
from datetime import datetime, timezone

with open('./newOutput.json','r') as f:
  data = json.load(f)

print(len(data))

1999


In [54]:
for i in data:
    
  timestamp = datetime.fromisoformat(i['post_timestamp'])

  hour = timestamp.hour
  if 0 <= hour < 6:
      time_of_day = 0
  elif 6 <= hour < 12:
      time_of_day = 1
  elif 12 <= hour < 18:
      time_of_day = 2
  elif 18 <= hour < 24:
      time_of_day = 3

  i['time_of_day'] = time_of_day
  i['day_of_week'] = timestamp.weekday()
  i['engagement'] = round((i['likes_count'] + i['comments']) / i['video_view_count'], 2)


In [55]:
# Extract engagement rates
engagement_rates = [item['engagement'] for item in data]

# Normalize engagement rates
min_engagement = min(engagement_rates)
max_engagement = max(engagement_rates)
normalized_engagement = [round((item['engagement'] - min_engagement) / (max_engagement - min_engagement), 2) for item in data]

# Update the 'engagement' values in the original dictionaries
for i, item in enumerate(data):
    if normalized_engagement[i] <= 0.33:
        item['engagement'] = 1
    elif 0.33 < normalized_engagement[i] <= 0.66:
        item['engagement'] = 2
    else:
        item['engagement'] = 3

print(data[0:2])

[{'post_timestamp': '2024-02-15 20:00:31', 'likes_count': 89956, 'video_view_count': 1162190, 'comments': 460, 'numHashtags': False, 'numTaggedUsers': True, 'sponsored': False, 'time_of_day': 3, 'day_of_week': 3, 'engagement': 1}, {'post_timestamp': '2024-03-09 14:00:00', 'likes_count': 206267, 'video_view_count': 1066268, 'comments': 604, 'numHashtags': False, 'numTaggedUsers': True, 'sponsored': False, 'time_of_day': 2, 'day_of_week': 5, 'engagement': 1}]


In [56]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score


# Extract features and target variable
X = np.array([[item['time_of_day'], item['day_of_week']] for item in data])
y = np.array([item['engagement'] for item in data])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=32)  # 80% training, 20% testing

# Train the Naive Bayes classifier
clf = CategoricalNB()
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.815
